# Расчет выйгрыша на каждом клиенте

In [10]:
import pandas as pd

import os
import os.path as os_path

cpfn = "diplom_project"

curr_dir = os.getcwd()
curr_dir = os_path.join(curr_dir[:curr_dir.find(cpfn)], cpfn)
data_storage = os_path.join(curr_dir, "data_storage")
input_data_fold = os_path.join(data_storage, "input_data_diplom")
int_rates_fold = os_path.join(
    os_path.join(os_path.join(curr_dir, "prog"), "interest rates"), "interest_rantes"
)

# данные о каждом из клиентов
data = pd.read_csv(
    input_data_fold + '.csv', 
    na_values = ['-', 'нет информации'],
    parse_dates = [
        'Автомобиль год выпуска1',
        'Автомобиль год выпуска2',
        'Автомобиль год выпуска3',
        'Дата регистрации договора',
        'Дата рождения',
        'Дата планируемого закрытия',
        'Дата фактического закрытия'
    ],
    low_memory = False
)

# данные о процентных ставках
interest_rates = pd.read_excel(
    int_rates_fold + ".xlsx",
    index_col = 0)

,до года,свыше года
2003-01-01,39.400000,39.500000
2003-02-01,38.900000,39.400000
2003-03-01,38.000000,39.600000
2003-04-01,37.100000,39.100000
2003-05-01,36.400000,38.100000
...,...,...
2020-08-01,11.747337,11.494557
2020-09-01,15.160468,11.649406
2020-10-01,16.872847,10.748919
2020-11-01,17.619200,10.061974


# Вычислительные функции

In [23]:
def get_int_rate(reg_date, term):
    '''Получение процентной ставки для выбранного клиента'''
    int_rate_ind = \
        (reg_date > interest_rates.index).sum()
    
    possible_rates = interest_rates.iloc[int_rate_ind - 1]
    
    if term < 12:
        rate = possible_rates[0]
    else:
        rate = possible_rates[1]
        
    return rate

def compute_loan_pay(loan_sum, rate, term):
    '''
        Получить размер одного платежа
        в случае равных платежей подобно тому как
        на сайте https://myfin.by/bank/belinvestbank/kredity/1362-kredit-na-tovary-otechestvennogo-proizvodstva?CreditsListCalcPotrebitelskieForm%5Bamount%5D=500&CreditsListCalcPotrebitelskieForm%5Bperiod%5D=3&CreditsListCalcPotrebitelskieForm%5Bpayment_type%5D=2&type_id=1&type=potrebitelskie&product_id=1362&app=&app-theme=#center=53.916782%2C27.555758&zoom=11&type=yandex%23map&balloon=
    '''
    # input:
    # loan_sum - занимаемая сумма
    # rate - ежемесячная ставка прцента
    # term - срок кредита в месяцах
    return (loan_sum*(1+rate)**term) / sum([(1+rate)**i for i in range(term)])



In [24]:
from datetime import date, timedelta
date(2021, 1,1) - timedelta(days = 5)

datetime.date(2020, 12, 27)

# Применяем вычисления ко всему множеству строк в DataFrame
Удалив только наблюдения, что имеют пропуск в "дата пранируемого закрытия" - это почти всегда "Овердрафт" и сохраняем результат.

In [26]:
data = data.dropna(subset = "Дата планируемого закрытия")

data['Срок кредита мес.'] = \
    (data["Дата планируемого закрытия"] - data["Дата регистрации договора"]).\
    astype('timedelta64[M]')

data['Процентная ставка'] = data.apply(
    lambda client_info: 
        get_int_rate(client_info["Дата регистрации договора"],
                     client_info["Срок кредита мес."]),
    axis = 1)


def comp_pay_count(client):
    '''
        Число проведенных платежей
    '''
    if client['Дефолт'] < 60:
        return client['Срок кредита мес.']
    get_month(client['Дата регистрации договора'], datetime.datetime(2021, 1, 1))
    
# data["Сумма к возврату"] = \
#     data['Сумма договора']*(1 + data['Процентная ставка']/36000)**\
#     data['Срок кредита дн.']


# # compute_profit(data.iloc[1])
# # data.iloc[1]["Сумма договора"]
# data["Сумма возврата"] = data.apply(compute_profit, axis = 1)

# data[["Сумма договора", "Сумма возврата"]].\
#     to_csv("/home/dranik/KFA/university/proc_ret_sum.csv")

In [50]:
import datetime

datetime.datetime(2021, 1, 1).month
#(datetime.datetime(2021, 1, 1).month - data.loc[0,'Дата регистрации договора'].month)
get_month = lambda start_date, end_date:\
    (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)



Timestamp('2016-01-04 00:00:00')